In [44]:
import numpy as np
import pandas as pd
import itertools

In [13]:
df_raw = pd.read_csv('/scratch-shared/martin/003_SPECS1K_ML/001_data/Specs935_ImageMeans_AfterQC_AnnotatedWithMOA.csv', sep=';')
df_raw.rename(columns={'Compound ID':'Compound_ID'}, inplace = True)

In [49]:
df = df_raw.copy()

In [50]:
df.dropna(subset=['Compound_ID'], inplace=True)
df = df[df['selected_mechanism'].str.contains('dmso')==False] # actually not dropping anything, since dropna already drop all dmso

In [51]:
# avg seems not good for GNN?

# averaging the image data based on Compound Id and mech

df_avg = df.groupby(['selected_mechanism','Compound_ID']).mean() # merge the data based on Compound_ID, avoiding reading "duplicated data"
df_avg.reset_index(inplace=True)
df_avg.drop('Compound_ID', axis=1, inplace=True)

In [52]:
df.drop(["Plate", "Plate_Well", "batch_id", "pertType", "Batch nr", "Compound_ID", "PlateID", "Well"], axis=1, inplace=True)

In [59]:
df

,ImageID,ImageNumber_nuclei,ObjectNumber_nuclei,Metadata_Site_nuclei,AreaShape_Area_nuclei,AreaShape_BoundingBoxArea_nuclei,AreaShape_BoundingBoxMaximum_X_nuclei,AreaShape_BoundingBoxMaximum_Y_nuclei,AreaShape_BoundingBoxMinimum_X_nuclei,AreaShape_BoundingBoxMinimum_Y_nuclei,...,RadialDistribution_ZernikePhase_illumSYTO_9_1_cytoplasm,RadialDistribution_ZernikePhase_illumSYTO_9_3_cytoplasm,RadialDistribution_ZernikePhase_illumSYTO_9_5_cytoplasm,RadialDistribution_ZernikePhase_illumSYTO_9_7_cytoplasm,RadialDistribution_ZernikePhase_illumSYTO_9_9_cytoplasm,Site,cmpd_conc,Flag,Count_nuclei,selected_mechanism
0,P03-L2_B03_1,1,64.536842,1,2467.273684,3451.578947,1148.673684,1053.315789,1089.421053,995.547368,...,0.137447,0.223677,-0.071381,-0.057143,-0.259786,1,10.0,0,109.0,estrogen receptor alpha modulator
1,P03-L2_B03_2,1,62.554455,2,2544.425743,3518.514851,1177.485149,986.623762,1118.435644,926.396040,...,-0.044756,0.063045,-0.133670,-0.070622,0.365823,2,10.0,0,121.0,estrogen receptor alpha modulator
2,P03-L2_B03_3,1,66.072165,3,2721.525773,3789.082474,963.505155,1203.257732,901.886598,1141.659794,...,0.071182,0.036852,-0.270887,-0.057549,-0.156699,3,10.0,0,118.0,estrogen receptor alpha modulator
3,P03-L2_B03_4,1,55.082353,4,2488.905882,3513.717647,1055.517647,1109.388235,995.623529,1050.811765,...,0.074981,0.205325,0.009515,-0.121825,0.119672,4,10.0,0,98.0,estrogen receptor alpha modulator
4,P03-L2_B03_5,1,57.574713,5,2711.804598,3772.517241,1065.724138,1151.264368,1003.747126,1089.965517,...,-0.019859,0.268856,-0.089310,-0.391028,-0.266201,5,10.0,0,100.0,estrogen receptor alpha modulator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54361,P12-L2_O23_5,1,57.494253,5,2543.252874,3586.540230,1005.804598,1075.827586,946.597701,1015.459770,...,0.039964,0.064709,0.041014,-0.214993,-0.386395,5,10.0,0,106.0,tubulin polymerization inhibitor
54362,P12-L2_O23_6,1,47.119403,6,2492.014925,3479.089552,1072.537313,1088.089552,1015.194030,1027.104478,...,-0.074372,-0.148499,0.158528,-0.079743,-0.343292,6,10.0,0,80.0,tubulin polymerization inhibitor
54363,P12-L2_O23_7,1,89.262774,7,2529.941606,3512.467153,938.007299,1030.525547,877.357664,972.218978,...,-0.198094,-0.226950,0.018661,-0.089651,0.242560,7,10.0,0,159.0,tubulin polymerization inhibitor
54364,P12-L2_O23_8,1,52.634146,8,2462.256098,3436.036585,1029.231707,1264.560976,971.536585,1204.536585,...,-0.542407,-0.072794,0.260223,0.270750,0.211333,8,10.0,0,97.0,tubulin polymerization inhibitor


In [60]:
node_features = df.iloc[:,1:-2]
labels = df.iloc[:,-1]

In [62]:
teams = df["Count_nuclei"].unique()
all_edges = np.array([], dtype=np.int32).reshape((0, 2))

In [64]:
for team in teams:
    team_df = df[df["Count_nuclei"] == team]
    players = team_df["ImageID"].values
    # Build all combinations, as all players are connected
    permutations = list(itertools.combinations(players, 2))
    edges_source = [e[0] for e in permutations]
    edges_target = [e[1] for e in permutations]
    team_edges = np.column_stack([edges_source, edges_target])
    all_edges = np.vstack([all_edges, team_edges])
# Convert to Pytorch Geometric format
edge_index = all_edges.transpose()
edge_index # [2, num_edges]

array([['P03-L2_B03_1', 'P03-L2_B03_1', 'P03-L2_B03_1', ...,
        'P12-L2_E22_4', 'P12-L2_E22_4', 'P12-L2_H13_1'],
       ['P03-L2_B03_8', 'P03-L2_B06_4', 'P03-L2_B07_1', ...,
        'P12-L2_H13_1', 'P12-L2_H13_5', 'P12-L2_H13_5']], dtype='<U32')

In [66]:
len(edge_index[0])

4422503